Scenario 2: A cross-functional team with one data scientist working on an ML model
MLflow setup:

tracking server: yes, local server
backend store: sqlite database
artifacts store: local filesystem
The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts_local

In [1]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [6]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/running_mlflow_examples/artifacts_local/1', creation_time=1720079251864, experiment_id='1', last_update_time=1720079251864, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1720079082831, experiment_id='0', last_update_time=1720079082831, lifecycle_stage='active', name='Default', tags={}>]

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/07/04 07:47:31 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: '/workspaces/mlops-zoomcamp/02-experiment-tracking/running_mlflow_examples/artifacts_local/1/2a783f9340d442129bf3c8b5517d430b/artifacts'


In [ ]:
# Interacting with the model registry

In [7]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5000")

In [9]:
mlflow.last_active_run()

<Run: data=<RunData: metrics={'accuracy': 0.96}, params={'C': '0.1', 'random_state': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "2a783f9340d442129bf3c8b5517d430b", '
                             '"artifact_path": "models", "utc_time_created": '
                             '"2024-07-04 07:47:32.108335", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.9.19", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.5.0", '
                             '"serialization_format": "cloudpickle", "code": '
                             'null}}, "model_uuid": '
                             '"3baa1836e87

In [15]:
mlflow.search_experiments()[0]

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/running_mlflow_examples/artifacts_local/1', creation_time=1720079251864, experiment_id='1', last_update_time=1720079251864, lifecycle_stage='active', name='my-experiment-1', tags={}>

In [17]:
runs = client.search_runs(experiment_ids=['1'])

if runs:
    run_id = runs[0].info.run_id

    # Register the model
    mlflow.register_model(
        model_uri=f"runs:/{run_id}/models",
        name='iris-classifier'
    )
else:
    print("No runs found for the experiment.")


mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/07/04 07:53:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.
Registered model 'iris-classifier' already exists. Creating a new version of this model...
2024/07/04 07:53:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 2
Created version '2' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1720079620951, current_stage='None', description='', last_updated_timestamp=1720079620951, name='iris-classifier', run_id='2a783f9340d442129bf3c8b5517d430b', run_link='', source='/workspaces/mlops-zoomcamp/02-experiment-tracking/running_mlflow_examples/artifacts_local/1/2a783f9340d442129bf3c8b5517d430b/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='2'>